# Docomposição do Marketing Share da ANP

# Setup Inicial

## Bibliotecas utilizadas

In [2]:
import pandas as pd
import numpy as np
import json

# Manipulação de datas
from time import strftime
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Visualização
import matplotlib.pyplot as plt
import plotly
import plotly.offline as py
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.graph_objs import *

## Definições Gerais

In [3]:
# definção dos diretórios relativos

# replace with your container name
blob_container_name = 'general' 

# replace with your relative folder path
blob_relative_path_raw = 'raw/mercado_potencial/'
blob_relative_path_enriched = 'enriched/mercado_potencial/'


# replace with your linked service name
linked_service_raw = 'LS_ADLS_RAW_01'
linked_service_enriched = 'LS_ADLS_ENRICHED_01'

ls_raw = mssparkutils.credentials.getPropertiesAll(linked_service_raw)
ls_enriched = mssparkutils.credentials.getPropertiesAll(linked_service_enriched)

converter_dic_raw = json.loads(ls_raw)
converter_dic_enriched = json.loads(ls_enriched)

end_point_raw = (converter_dic_raw['Endpoint'].split("/"))[2]
end_point_enriched = (converter_dic_enriched['Endpoint'].split("/"))[2]


dir_raw = 'abfss://%s@%s/%s' % (blob_container_name, end_point_raw, blob_relative_path_raw)
dir_enriched = 'abfss://%s@%s/%s' % (blob_container_name, end_point_enriched, blob_relative_path_enriched)


In [4]:
# dir_row  = 'abfss://general@stedlk01dtandev.dfs.core.windows.net/raw/mercado_potencial/'
# dir_enriched = 'abfss://general@stedlk02dtandev.dfs.core.windows.net/enriched/mercado_potencial/'

# ls_row = 'LS_ADLS_ROW_01'
# ls_enriched = 'LS_ADLS_ENRICHED_01'

## Funções auxiliares

In [5]:
def ajustar_data(df):
    """Cria data a partir de dados de ano e mês separados.

        Ajusta e cria coluna em dataframe a partir de informações de 
        ano e mês separadas e em formato string. 
        
    Parâmetros
    ----------
    df: pd.DdataFrame
        Dataframe contendo as colunas 'Ano de MESANO' e 'Num mês'
    
    Retorno
    -------
    df: pd.DdataFrame
        Dataframe contendo a coluna 'Data' em formato datatime
    """

    df['Data'] = pd.to_datetime(['-'.join(i) for i in zip(df['Ano de MESANO'].map(str),df['Num mês'].map(str))],format='%Y-%m')
    df.sort_values(by='Data', inplace=True)
    df = df.set_index('Data', drop=True)
    return (df)

In [6]:
def grafico_diff (df, titulo=""):
    """Cria gráficos de linhas para as diferenças entre ANP e VIBRA.

        Cria gráficos de linhas para as diferenças entre os dados 
        da ANP e do histórico de vendas da VIBRA 
        
    Parâmetros
    ----------
    df: pd.DdataFrame
        Dataframe contendo as colunas 'data_venda' e 'CONSUMIDOR FINAL', sendo esta útima 
        a diferença entre os dados de volume de vendas fornecidos pela ANP e os 
        registrados por Histórico de vendas da VIBRA
    
    Retorno
    -------
    fig: plotly object
        Grafico em formato plotly que poderá ser exibido ou salvo.
    """
    
    df.reset_index(inplace=True)

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=df['data_venda'],
        y=df['CONSUMIDOR FINAL'],
        line={'color': 'rgb(220,20,60)'},
        name='Diferença'
    ))

    fig.update_layout(
        title="Diferença "+ titulo +" entre o volumes de venda de Diesel <br><sup>entre ANP e Vendas Vibra para o Consumidor Final<sup>",
        xaxis_title="Período",
        #yaxis_title="Volume (m³)",
        legend_title="Legenda",
        font=dict(
            family="Courier New, monospace",
            size=14,
            color="royalblue"
        )
    )
    
    return (fig)

# Análise Exploratória

## Coletando dados de vendas Segmentados por Distribuidoras e por Cliente

In [7]:
# este arquivo é fornecido periodicamente pelo Sérgio
# tando na sandbox quanto para TI por em PRD
arquivo = dir_raw + 'volumetria/Dados_MarketShare_CONS_TRR.xlsx' 

mkt_cf = pd.read_excel(arquivo, sheet_name='CONSUMIDOR - %', 
                        storage_options = {'linked_service' : linked_service_raw})
mkt_trr = pd.read_excel(arquivo, sheet_name='TRR - %',
                        storage_options = {'linked_service' : linked_service_raw})

mktcf = pd.read_excel(arquivo, sheet_name='CONSUMIDOR - Volumes',
                        storage_options = {'linked_service' : linked_service_raw})
mkttrr = pd.read_excel(arquivo, sheet_name='TRR - Volumes',
                        storage_options = {'linked_service' : linked_service_raw})

## Ajuste de Datas

In [8]:
mkt_cf = ajustar_data(mkt_cf)
mkt_trr = ajustar_data(mkt_trr)

mktcf = ajustar_data(mktcf)
mkttrr = ajustar_data(mkttrr)

## Evolução do Market Share ao longo dos anos

In [9]:
empresas = np.unique(mkt_trr['_EMPRESA'].values)
cores = ['green', 'purple', 'blue', 'red']

fig = make_subplots(rows=2, cols=1,
                        y_title='% Mercado',
                        subplot_titles=('Consumidor Final', 'TRR'),
                        shared_yaxes=False,
                        vertical_spacing=0.1 )

for c, i in enumerate(empresas[::-1]):
# linhas do Consumidor Final
        dt = mkt_cf[mkt_cf['_EMPRESA']==i]['Valores de medida']
        fig.add_trace(go.Scatter(
                        name = i,
                        x = dt.index,
                        y = dt.values*100,
                        mode='lines+markers',
                        marker={'color': cores[c],
                                'size': 4,
                                'line': {'color': cores[c],
                                        'width': .65}
                                },
                        visible=True),
                    row=1, col=1
        )
# linhas do TRR
        dt = mkt_trr[mkt_trr['_EMPRESA']==i]['Valores de medida']
        fig.add_trace(go.Scatter(
                        name = i,
                        x = dt.index,
                        y = dt.values*100,
                        mode='lines+markers',
                        marker={'color': cores[c],
                                'size': 4,
                                'line': {'color': cores[c],
                                        'width': .65}
                                },
                        visible=True,
                        showlegend=False),
                    row=2, col=1
        )
        
fig.add_annotation(
                showarrow=False,
                text='Gerado em ' + datetime.now().strftime("%d %b %Y às %H:%M:%S"),
                font=dict(size=12),
                xref='paper',
                x=1.1,
                yref='paper',
                y=-0.25)
    
fig.update_xaxes(matches='x', rangeslider_visible=True, rangeslider_thickness = 0.08)
fig.update_xaxes(rangeslider= {'visible':False}, row=1, col=1)

ano_s = dt.index.min().strftime('%m.%Y')
ano_e = dt.index.max().strftime('%m.%Y')

fig.update_layout(
                title = 'Market Share Diesel Brasil<br><sup>('+ ano_s +' - '+ ano_e+ ')</sup>',
                #xaxis_title = "Período",
                #yaxis_title = "% mercado",
                legend_title = "Legenda",
                font=dict(
                    family = "Courier New, monospace",
                    size = 14,
                    color = "royalblue"),
                    height=800)
fig = plotly.offline.plot(fig, output_type='div')
displayHTML(fig)
        

# Decomposição do Consumidor Final
Desagregação do mkt share consumidor final da ANP por Atividades Econômicas VIBRA por meio da aplicação das proporções oriundas do histórico de vendas VIBRA

## GET Histórico Vendas VIBRA por Atividades Econômicas

In [10]:
prop  = pd.read_parquet(dir_enriched+'volumetria/vol01_prop_consfinal_vegetativo.parquet',
                        storage_options = {'linked_service' : linked_service_enriched})
                        
cf  = pd.read_parquet(dir_enriched+'volumetria/vol01_vendas_vibra_mes.parquet',
                        storage_options = {'linked_service' : linked_service_enriched})

## GET Volume Vendas Consumidor Final Vibra MKTShare 

In [11]:
mktcf_vibra = pd.DataFrame(mktcf[mktcf['_EMPRESA']=='VIBRA']['Valores de medida'])
mktcf_vibra.columns = ['Vol_ConsFinal']
mktcf_vibra = mktcf_vibra['2016':]

## Volume de Vendas Consumidor Final Vibra MKTShare Decomposta por Atividade Econômica

In [12]:
# volume por atividade economica decomposta a partir das proporções de vendas vibra

beg = mktcf_vibra.index.min().strftime("%Y-%m-%d")
end = mktcf_vibra.index.max().strftime("%Y-%m-%d")
# A multiplicação sempre estará delimitada ao periodo do MKT SHARE

vol = prop.loc[beg:end].mul(mktcf_vibra['Vol_ConsFinal'].values, axis=0)

## Salvando Volume decomposto

In [13]:
vol.to_parquet(dir_enriched + 'volumetria/vol02_MKTS_ConsFinal_Decomposto.parquet', 
                      storage_options = {'linked_service':linked_service_enriched})

## Diferença entre os volumes de venda Consumidor Final Vibra e ANP

In [14]:
diff = cf['CONSUMIDOR FINAL'][beg:end].sub(mktcf_vibra['Vol_ConsFinal'].values, axis=0) # diferença mensal
diff = pd.DataFrame(diff)
diff_a = diff.resample('Y').sum()#.reset_index(drop=False) # Ano


In [15]:
grafico_diff(diff, 'mensal').show()

Diferenças entre os volumes da Vibra e o registrado na ANP parece sofrer ajustes entre os meses de Dezembro e Janeiro

In [16]:
grafico_diff(diff_a, 'anual').show()

Diferença entre o volume informado pela empresa e o disponibilizado pela ANP vem se estabilizando próximo de valores mínimos 

# ACESSO À DOCUMENTAÇÃO

In [17]:
help(ajustar_data)

In [18]:
help(grafico_diff)